In [17]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from eden.converter.molecule import obabel
import networkx as nx
import pybel
import requests
import os.path

In [19]:
%%time
AID=1
#AID=2401
#DATA_DIR = '/Volumes/seagate/thesis/examples/data'
DATA_DIR = '/Users/jl/uni-freiburg/thesis/EDeN/examples/3Dmodel/data'
active_fname=DATA_DIR + '/AID%s_active.smi'%AID
inactive_fname=DATA_DIR + '/AID%s_inactive.smi'%AID

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 16.9 µs


In [20]:
active_fname

'/Users/jl/uni-freiburg/thesis/EDeN/examples/3Dmodel/data/AID1_active.smi'

In [21]:
def make_iterable(filename):
    with open(filename) as f:
        for line in f:
            yield line.strip()

Functions for training and testing the model

In [22]:
import datetime, time
def train_obabel_model(iterable_pos, iterable_neg, data_dir,
                       model_type = "default",
                       model_fname=None, n_iter=40, active_set_size=1000,
                       n_active_learning_iterations=3, threshold=1, train_test_split=0.7,
                       verbose=False):

    from numpy.random import randint
    from numpy.random import uniform


    global_cache = {}

    # this will be passed as an argument to the model later on
    def pre_processor(data, model_type="3d", **kwargs):

        #### Use the model_type variable from outside (?) ####
        # model_type = kwargs.get('mode', 'default')
        if model_type == "default":
            iterable = obabel.obabel_to_eden(data, **kwargs)
        elif model_type == "3d":
            iterable = obabel.obabel_to_eden3d(data, cache=global_cache, **kwargs)
        return iterable

    from eden.graph import Vectorizer
    vectorizer = Vectorizer()

    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(class_weight='auto', shuffle=True)

    #######3
    #create iterable from files
    ########

    from itertools import tee
    iterable_pos, iterable_pos_ = tee(iterable_pos)
    iterable_neg, iterable_neg_ = tee(iterable_neg)

    import time
    start = time.time()
    print('# positives: %d  # negatives: %d (%.1f sec %s)'%(sum(1 for x in iterable_pos_), sum(1 for x in iterable_neg_), time.time() - start, str(datetime.timedelta(seconds=(time.time() - start)))))

    iterable_pos, iterable_pos_ = tee(iterable_pos)
    iterable_neg, iterable_neg_ = tee(iterable_neg)

    #split train/test
    from eden.util import random_bipartition_iter
    iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
    iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)



    #make predictive model
    from eden.model import ActiveLearningBinaryClassificationModel
    model = ActiveLearningBinaryClassificationModel(pre_processor,
                                                    estimator=estimator,
                                                    vectorizer=vectorizer,
                                                    n_jobs=1,
                                                    n_blocks = 2,
                                                    fit_vectorizer=True)

    #optimize hyperparameters and fit model

    pre_processor_parameters={'k':randint(1, 10,size=n_iter),
                             'threshold':randint(1, 10, size=n_iter),
                             'model_type':[model_type]}

    #print "pre processor parameters: " + str(pre_processor_parameters)
    vectorizer_parameters={'complexity':[2,3,4],
                           'discretization_size':randint(2, 3,size=n_iter),
                           'discretization_dimension':randint(2, 3,size=n_iter)}

    estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                          'penalty':['l1','l2','elasticnet'],
                          'l1_ratio':uniform(0.1,0.9, size=n_iter),
                          'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                          'power_t':uniform(0.1, size=n_iter),
                          'alpha': [10**x for x in range(-8,-2)],
                          'eta0': [10**x for x in range(-4,-1)],
                          'learning_rate': ["invscaling", "constant", "optimal"]}

    print "calling optimizer.."
    model.optimize(iterable_pos_train, iterable_neg_train,
                   model_name=model_fname,
                   n_active_learning_iterations=n_active_learning_iterations,
                   size_positive=-1,
                   size_negative=active_set_size,
                   n_iter=n_iter, cv=3, verbose=verbose,
                   pre_processor_parameters=pre_processor_parameters,
                   vectorizer_parameters=vectorizer_parameters,
                   estimator_parameters=estimator_parameters)

    #estimate predictive performance
    #model.estimate( iterable_pos_test, iterable_neg_test, cv=5 )
    # Had to change this call, estimate has no cv parameter
    model.estimate( iterable_pos_test, iterable_neg_test )

    return model

def test_obabel_model(fname, model_type = "default", model_fname=None):
    from eden.model import ActiveLearningBinaryClassificationModel

    model = ActiveLearningBinaryClassificationModel()
    model.load(model_fname)

    #create iterable from files
    from eden.converter.molecule import obabel
    if model_type == "default":
        iterable=obabel.obabel_to_eden(fname)
    elif model_type == "3d":
        iterable=obabel.obabel_to_eden3d(fname)

    predictions= model.decision_function( iterable )

    return predictions

Train the models

**3D model**
---

In [23]:
#%%time
pos_iterator=make_iterable(active_fname) #this is a SMILES file
neg_iterator=make_iterable(inactive_fname) #this is a SMILES file

model_fname=DATA_DIR + '/AID%s.model3d'%AID

model = train_obabel_model(pos_iterator, neg_iterator,
                           data_dir=DATA_DIR,
                           model_type = "3d",
                           model_fname=model_fname,
                           n_iter=5,
                           active_set_size=5,
                           n_active_learning_iterations=0,
                           threshold=1,
                           train_test_split=0.8,
                           verbose=1)

# positives: 10  # negatives: 20 (0.0 sec 0:00:00.000308)
calling optimizer..

Iteration: 1/5 (at 2270.1 sec; 0:37:50.058105)
Best score (roc_auc): 1.000000 (1.000000 +- 0.000000)
Instances: 15 ; Features: 1048577 with an avg of 845 features per instance
class: 1 count:5 (0.33)	class: -1 count:10 (0.67)	
Classifier:
SGDClassifier(alpha=0.0001, average=False, class_weight='auto', epsilon=0.1,
       eta0=0.001, fit_intercept=True, l1_ratio=0.86345999900080306,
       learning_rate='optimal', loss='modified_huber', n_iter=46, n_jobs=1,
       penalty='l1', power_t=0.54162985685335086, random_state=None,
       shuffle=True, verbose=0, warm_start=False)
--------------------------------------------------------------------------------
Instances: 15 ; Features: 1048577 with an avg of 860 features per instance
--------------------------------------------------------------------------------
Test Estimate
             precision    recall  f1-score   support

         -1       1.00      1.00    

In [35]:
from eden.model import ActiveLearningBinaryClassificationModel
from eden.util import random_bipartition_iter

AID=2401

pos_iterator=make_iterable(active_fname) #this is a SMILES file
neg_iterator=make_iterable(inactive_fname) #this is a SMILES file
train_test_split = 0.8
iterable_pos_train, iterable_pos_test = random_bipartition_iter(pos_iterator, relative_size=train_test_split)
iterable_neg_train, iterable_neg_test = random_bipartition_iter(neg_iterator, relative_size=train_test_split)

model_fname=DATA_DIR + '/AID%s.model3d'%AID
model = ActiveLearningBinaryClassificationModel()
model.load(model_fname)
model.estimate( iterable_pos_test, iterable_neg_test )


Classifier:
SGDClassifier(alpha=0.0001, average=False, class_weight='auto', epsilon=0.1,
       eta0=0.001, fit_intercept=True, l1_ratio=0.6307434669087042,
       learning_rate='optimal', loss='modified_huber', n_iter=78, n_jobs=1,
       penalty='l1', power_t=0.34898171101291842, random_state=None,
       shuffle=True, verbose=0, warm_start=False)
--------------------------------------------------------------------------------
[<networkx.classes.graph.Graph object at 0x10b9cb610>, <networkx.classes.graph.Graph object at 0x10b9cb990>]


ZeroDivisionError: integer division or modulo by zero

In [33]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_active_learning_data_matrices',
 '_assemble_data_matrix',
 '_bounded_selection',
 '_data_matrices',
 '_data_matrix',
 '_sample',
 'decision_function',
 'description',
 'estimate',
 'estimator',
 'estimator_args',
 'fit',
 'fit_vectorizer',
 'get_description',
 'get_estimator',
 'get_info',
 'get_pre_processor',
 'get_vectorizer',
 'info',
 'load',
 'n_blocks',
 'n_jobs',
 'optimize',
 'pre_processor',
 'pre_processor_args',
 'predict',
 'print_model_parameter_configuration',
 'save',
 'vectorizer',
 'vectorizer_args']

In [30]:
if 0:
    pos_iterator=make_iterable(active_fname) #this is a SMILES file
    neg_iterator=make_iterable(inactive_fname) #this is a SMILES file 
    global_cache={}
    def pre_processor(data, model_type="default", converter=None, **kwargs):

            #### Use the model_type variable from outside (?) ####
            # model_type = kwargs.get('mode', 'default')
            from itertools import tee, islice
            from eden.util.display import serialize_graph
            data, data_ = tee(data)
            for x in data_:
                print x
            if model_type == "default":
                iterable = obabel.obabel_to_eden(data, **kwargs)
            elif model_type == "3d":
                iterable = obabel.obabel_to_eden3d(data, cache=global_cache, **kwargs)
            iterable_, iterable = tee(iterable)

            #for x in islice(iterable_, 1):
                #print serialize_graph(x)
            return iterable

    pos_graphs = pre_processor(pos_iterator, model_type="3d")
    neg_graphs = pre_processor(neg_iterator, model_type="3d")
    from eden.graph import Vectorizer
    vectorizer = Vectorizer(discretization_size = 2, discretization_dimension=2)
    from eden import vectorize
    X_pos = vectorizer.fit_transform(pos_graphs)
    X_neg = vectorizer.fit_transform(neg_graphs)
    print X_pos
    print X_neg

Test the models:

In [ ]:
# active_X, inactive_X are created as above (data matrices) with vectorize(...)
active_X
inactive_X
from eden.util import fit
fit(active_X, inactive_X, vectorizer)
# do transform on both, fit only on positive
### next meet wednesday 16.30